In [17]:
import pandas as pd
import numpy as np
import importlib
import gc
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # Cambiamos el import
from prophet import Prophet
import sys
sys.path.append('./scripts')  
import feature_engineering
importlib.reload(feature_engineering)

<module 'feature_engineering' from 'c:\\Users\\Usuario\\Documents\\Universidad\\austral\\2025\\Lab3\\Lab3-MCD\\notebooks\\entregable\\./scripts\\feature_engineering.py'>

In [18]:
ESTOY_EN_KAGGLE = True

In [19]:
if ESTOY_EN_KAGGLE:
    df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado_201912.csv", sep=',', encoding='utf-8')
else:
    df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado.csv", sep=',', encoding='utf-8')
df

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.576908,NaN,0.461891,-1.704215,-0.465448,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-2.042103,NaN,-0.046911,-2.039835,-1.325726,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.323111,NaN,0.786658,-0.435961,0.267471,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.117043,NaN,-1.703222,-1.074260,0.333316,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.353285,NaN,2.865166,0.273781,-1.470536,7.315352,NaN,6.553933,7.347100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,1.850294,0.000000,NaN,NaN,NaN
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,-0.707610,0.000000,NaN,NaN,NaN
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,1.0,2.237248,NaN,1.414214,1.397345,0.272537,0.087406,NaN,2.197225,0.100198
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [20]:
#### Revisar si hay infinitos en la columna target
print("Valores infinitos en 'tn_zscore':", np.isinf(df['tn_zscore']).sum())
# Revisar también NaN
print("Valores NaN en 'tn_zscore':", df['tn_zscore'].isna().sum())
df['tn_zscore'] = df['tn_zscore'].replace([np.inf, -np.inf], np.nan)
# df['tn_zscore'] = df['tn_zscore'].fillna(df['tn_zscore'].median())  # Usar la mediana
df['tn_zscore'] = df['tn_zscore'].fillna(0)  # Usar cero #### EXPERIMENTAR CON ESTO
df

Valores infinitos en 'tn_zscore': 0
Valores NaN en 'tn_zscore': 0


,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.576908,NaN,0.461891,-1.704215,-0.465448,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-2.042103,NaN,-0.046911,-2.039835,-1.325726,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.323111,NaN,0.786658,-0.435961,0.267471,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.117043,NaN,-1.703222,-1.074260,0.333316,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.353285,NaN,2.865166,0.273781,-1.470536,7.315352,NaN,6.553933,7.347100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,1.850294,0.000000,NaN,NaN,NaN
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,-0.707610,0.000000,NaN,NaN,NaN
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,1.0,2.237248,NaN,1.414214,1.397345,0.272537,0.087406,NaN,2.197225,0.100198
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [5]:
df[df['product_id'] == 20001]

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.576908,NaN,0.461891,-1.704215,-0.465448,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-2.042103,NaN,-0.046911,-2.039835,-1.325726,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.323111,NaN,0.786658,-0.435961,0.267471,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.117043,NaN,-1.703222,-1.074260,0.333316,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.353285,NaN,2.865166,0.273781,-1.470536,7.315352,NaN,6.553933,7.347100
5,20001,201706,201701,201912,6,36,0,0,3000.0,NaN,...,0.0,0.414052,NaN,1.447017,0.354905,-0.598005,7.327166,NaN,6.347389,7.363171
6,20001,201707,201701,201912,7,36,0,0,3000.0,NaN,...,0.0,-1.250684,NaN,-0.599015,-1.224227,-0.415363,6.938938,NaN,5.945421,6.991610
7,20001,201708,201701,201912,8,36,0,0,3000.0,NaN,...,0.0,-0.445444,NaN,2.237283,-0.568519,0.037302,7.145507,NaN,6.467699,7.162916
8,20001,201709,201701,201912,9,36,0,0,3000.0,NaN,...,0.0,-0.276888,NaN,-0.599015,-0.351370,0.555977,7.183830,NaN,5.945421,7.213740
9,20001,201710,201701,201912,10,36,0,0,3000.0,NaN,...,0.0,0.140867,NaN,-1.768175,-0.078236,-1.401555,7.272923,NaN,5.613128,7.274204


In [21]:
##### Aca puedo agregar todas las columnas que quiero usar como features 

#### cuidado con el data leakage, no usar el target en el entrenamiento
df = feature_engineering.get_lags(df, "tn_zscore", 201912)
df = feature_engineering.get_delta_lags(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_means(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_stds(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_mins(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn_zscore", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_medians(df, "tn_zscore", 201912)



#  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

#  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty_zscore')


WARNING - (py.warnings._showwarnmsg) - c:\Users\Usuario\.conda\envs\py311lab3\Lib\site-packages\pandas\core\window\rolling.py:1535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return window_func(values, begin, end, min_periods)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Usuario\.conda\envs\py311lab3\Lib\site-packages\pandas\core\window\rolling.py:1535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return window_func(values, begin, end, min_periods)

WARNING - (py.warnings._showwarnmsg) - c:\Users\Usuario\.conda\envs\py311lab3\Lib\site-packages\pandas\core\window\rolling.py:1535: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results 

In [22]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)

In [8]:
# ##### TARDA 66 MINUTOS EN EJECUTARSE
# columns = ['tn_zscore', 'stock_final_zscore', 'cust_request_qty_zscore', 'cust_request_tn_zscore']

# if ESTOY_EN_KAGGLE:
#     df = feature_engineering.get_prophet_features(df, 'tn_zscore') 
#     df = feature_engineering.get_prophet_features(df, 'stock_final_zscore') 
#     df = feature_engineering.get_prophet_features(df, 'cust_request_qty_zscore') 
#     df = feature_engineering.get_prophet_features(df, 'cust_request_tn_zscore') 
# else:
#     df = feature_engineering.get_prophet_features_sin_data_leakage(df, 'tn_zscore', train_mask = (df['periodo'] <= 201909))

In [ ]:
# df = feature_engineering.get_neural_prophet_features(df)

In [37]:
df[df['product_id'] == 20060].filter(regex='^(?!cat)')

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,dia_del_year,dia_del_year_sin,dia_del_year_cos,ds,trend_add,yearly_add,additive_terms,trend_mult,yearly_mult,multiplicative_terms
2079,20060,201701,201701,201905,1,29,0,0,800.0,NaN,...,1,0.017213,0.999852,2017-01-01,1.670359,-1.780342,-1.780342,1.670359,-1.706775,-1.706775
2080,20060,201702,201701,201905,2,29,0,0,800.0,NaN,...,32,0.523416,0.852078,2017-02-01,1.548531,-1.558282,-1.558282,1.548531,-1.757946,-1.757946
2081,20060,201703,201701,201905,3,29,0,0,800.0,NaN,...,60,0.858764,0.512371,2017-03-01,1.438492,0.958228,0.958228,1.438492,0.050403,0.050403
2082,20060,201704,201701,201905,4,29,0,0,800.0,NaN,...,91,0.999991,0.004304,2017-04-01,1.316664,0.259218,0.259218,1.316664,-0.049612,-0.049612
2083,20060,201705,201701,201905,5,29,0,0,800.0,NaN,...,121,0.871706,-0.490029,2017-05-01,1.198766,0.544768,0.544768,1.198766,-0.649513,-0.649513
2084,20060,201706,201701,201905,6,29,0,0,800.0,NaN,...,152,0.501242,-0.865307,2017-06-01,1.076938,0.237052,0.237052,1.076938,-0.036208,-0.036208
2085,20060,201707,201701,201905,7,29,0,0,800.0,NaN,...,182,0.008607,-0.999963,2017-07-01,0.959040,1.793749,1.793749,0.959040,1.470718,1.470718
2086,20060,201708,201701,201905,8,29,0,0,800.0,NaN,...,213,-0.501242,-0.865307,2017-08-01,0.837212,-0.171366,-0.171366,0.837212,-0.205690,-0.205690
2087,20060,201709,201701,201905,9,29,0,0,800.0,NaN,...,244,-0.871706,-0.490029,2017-09-01,0.715384,1.761853,1.761853,0.715384,2.007232,2.007232
2088,20060,201710,201701,201905,10,29,0,0,800.0,NaN,...,274,-0.999991,0.004304,2017-10-01,0.597486,-0.223172,-0.223172,0.597486,-0.101224,-0.101224


In [41]:
df[df['product_id'] == 20001].filter(regex='_rolling_mean_')

,tn_zscore_rolling_mean_1,tn_zscore_rolling_mean_2,tn_zscore_rolling_mean_3,tn_zscore_rolling_mean_4,tn_zscore_rolling_mean_5,tn_zscore_rolling_mean_6,tn_zscore_rolling_mean_7,tn_zscore_rolling_mean_8,tn_zscore_rolling_mean_9,tn_zscore_rolling_mean_10,...,tn_zscore_rolling_mean_27,tn_zscore_rolling_mean_28,tn_zscore_rolling_mean_29,tn_zscore_rolling_mean_30,tn_zscore_rolling_mean_31,tn_zscore_rolling_mean_32,tn_zscore_rolling_mean_33,tn_zscore_rolling_mean_34,tn_zscore_rolling_mean_35,tn_zscore_rolling_mean_36
0,-1.492929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.941219,-1.717074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.284694,-1.112957,-1.239614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.049775,-0.667235,-1.091896,-1.192154,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.367121,-0.341327,-0.322449,-0.727142,-0.880299,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.425680,0.396400,-0.085658,-0.135417,-0.496578,-0.662636,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-1.178560,-0.376440,-0.128586,-0.358884,-0.344046,-0.610241,-0.736339,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.402583,-0.790571,-0.385154,-0.197085,-0.367623,-0.353802,-0.580576,-0.694620,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.240151,-0.321367,-0.607098,-0.348904,-0.205699,-0.346378,-0.337566,-0.538023,-0.644123,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.162422,-0.038865,-0.160104,-0.414718,-0.246638,-0.144345,-0.273692,-0.275068,-0.460196,-0.563469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,tn_lag_12_season_adj_add,tn_lag_12_season_adj_mul,tn_lag_12_season_adj_add_norm,tn_lag_12_season_adj_mul_norm,dtw_cluster,dist_to_centroid,simil_to_top,corr_tn_dolar,corr_tn_ipc,corr_tn_dolar_x_prod
0,20001,201701,201701,201912,1,36,0,0,3000.0,0.0,...,0.0,0.0,0.0,0.0,41,3.489215,0.000000,NaN,NaN,0.343657
1,20001,201702,201701,201912,2,36,0,0,3000.0,0.0,...,0.0,0.0,0.0,0.0,41,3.489215,0.000000,NaN,NaN,0.343657
2,20001,201703,201701,201912,3,36,0,0,3000.0,0.0,...,0.0,0.0,0.0,0.0,41,3.489215,0.000000,NaN,NaN,0.343657
3,20001,201704,201701,201912,4,36,0,0,3000.0,0.0,...,0.0,0.0,0.0,0.0,41,3.489215,0.000000,NaN,NaN,0.343657
4,20001,201705,201701,201912,5,36,0,0,3000.0,0.0,...,0.0,0.0,0.0,0.0,41,3.489215,0.000000,NaN,NaN,0.343657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,0.0,0.0,...,0.0,0.0,0.0,0.0,37,2.317218,2258.537821,-0.125809,-0.067496,0.204382
31358,21281,201705,201702,201708,4,7,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,37,2.317218,2258.537821,-0.098866,-0.047311,0.204382
31359,21281,201706,201702,201708,5,7,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,37,2.317218,2258.537821,-0.377332,-0.420269,0.204382
31360,21281,201707,201702,201708,6,7,1,0,0.0,0.0,...,0.0,0.0,0.0,0.0,37,2.317218,2258.537821,-0.337101,-0.376122,0.204382


In [ ]:
df = feature_engineering.get_anomaliasPoliticas(df)
df = feature_engineering.get_IPC(df)
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_mes_receso_escolar(df)
df = feature_engineering.mes_con_feriado(df)
df = feature_engineering.chatGPT_features_serie(df, "tn_zscore")
df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df = feature_engineering.dwt_features_serie(df, cant_clusters=50) # df = feature_engineering.get_clustersDTW(df, n_clusters=50)

df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)

##### Levantar csv: Prophet y Neural Prophet

./datasets/features_neuralprophet_completo.csv

./datasets/prophet_features_tn_zscore.csv

./datasets/prophet_features_cust_request_qty_zscore.csv

./datasets/prophet_features_cust_request_tn_zscore.csv

./datasets/prophet_features_stock_final_zscore.csv

In [ ]:
##### LEVANTAR LOS CSV  y AGREGARLOS AL DF

In [27]:
if ESTOY_EN_KAGGLE:
    df.to_csv("./datasets/periodo_x_producto_con_target_transformado_con_feature_engineering_201912.csv", index=False, sep=',', encoding='utf-8')
else:
    df.to_csv("./datasets/periodo_x_producto_con_target_transformado_con_feature_engineering.csv", index=False, sep=',', encoding='utf-8')